In [1]:
import pandas as pd
df = pd.read_csv("Dataset_Warren_Buffet_Clean.csv")
print(df.head(5))

                                            question  \
0      Why do you emphasize psychology of investing?   
1  What role does purchasing undervalued business...   
2  How does valuation cushion influence your inve...   
3          How do you think about valuation cushion?   
4                         Why do you emphasize moat?   

                                              answer               category  \
0  Speaking for myself Investing success depends ...     Behavioral Finance   
1  I practice value investing by buying businesse...  Fundamental Valuation   
2  I feel A margin of safety means buying assets ...  Fundamental Valuation   
3  I believe A margin of safety means buying asse...  Fundamental Valuation   
4  For my part i seek businesses with wide econom...          Uncategorized   

      refined_category  
0  Investor Psychology  
1      Intrinsic Value  
2      Intrinsic Value  
3      Intrinsic Value  
4       Economic Moats  


In [2]:
from sentence_transformers import SentenceTransformer
embedder = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [3]:
questions = df['question'].tolist()
question_embeddings = embedder.encode(questions, convert_to_numpy=True)

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [4]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 54.2 MB/s eta 0:00:00


In [5]:
import faiss
dim = question_embeddings.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(question_embeddings)
print(f"Indexed {index.ntotal} vectors.")

Indexed 4996 vectors.


In [6]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.7 MB/s eta 0:00:00


In [7]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
emb_func = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vector_db = FAISS.from_texts(texts=questions, embedding=emb_func)

/tmp/ipython-input-13382192.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  emb_func = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [8]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
emb_func = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vector_db = FAISS.from_texts(texts=questions, embedding=emb_func)

In [9]:
def retrieve_docs(query, top_k=1):
    # Step 1: embed the query
    q_vec = embedder.encode([query], convert_to_numpy=True)
    # Step 2: search the FAISS index
    distances, indices = index.search(q_vec, top_k)
    # Step 3: retrieve the top_k answers
    results = []
    for idx in indices[0]:
        qa_pair = df.iloc[idx]  # get the Q&A from original data
        results.append({
            "question": qa_pair['question'],
            "answer": qa_pair['answer']
        })
    return results

# Example usage:
query = "Why is having a margin of safety important?"
matches = retrieve_docs(query, top_k=2)
for m in matches:
    print("Match Q:", m['question'], "\nMatch A:", m['answer'], "\n")

Match Q: Why do you believe margin of safety is important? 
Match A: In light of margin of safety, a margin of safety means buying assets at a discount to their intrinsic value to minimize the risk of loss 

Match Q: Why do you believe safety margin is important? 
Match A: On the subject of safety margin, a margin of safety means buying assets at a discount to their intrinsic value to minimize the risk of loss 



In [10]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [11]:
retrieved = retrieve_docs(query, top_k=1)
context_text = retrieved[0]['answer']  # taking the top answer text
prompt = f"Context: {context_text}\n\nQuestion: {query}\nAnswer:"

In [12]:
inputs = tokenizer(prompt, return_tensors='pt')
outputs = model.generate(**inputs, max_length=200)
answer_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(answer_text)

minimize the risk of loss


In [13]:
from transformers import T5ForConditionalGeneration, TrainingArguments, Trainer
from datasets import Dataset

# Format inputs/targets
df['prompt'] = "question: " + df['question']
df['response'] = df['answer']

# Convert to Hugging Face Dataset
dataset = Dataset.from_pandas(df[['prompt', 'response']])

# Load tokenizer and model again (reuse if already loaded)
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")

# Tokenize data
MAX_LEN = 256
def preprocess(example):
    model_inputs = tokenizer(example['prompt'], max_length=MAX_LEN, truncation=True, padding='max_length')
    labels = tokenizer(example['response'], max_length=MAX_LEN, truncation=True, padding='max_length')
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

tokenized_dataset = dataset.map(preprocess)

# Define training args
training_args = TrainingArguments(
    output_dir="./buffett_flan_model",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    learning_rate=5e-5,
    logging_dir="./logs",
    save_strategy="epoch",
    save_total_limit=2,
    weight_decay=0.01,
    report_to="none"
)

# Train the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
)

trainer.train()


Map:   0%|          | 0/4996 [00:00<?, ? examples/s]

/tmp/ipython-input-1102756423.py:39: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
500,1.778200
1000,0.135600


Step,Training Loss
500,1.778200
1000,0.135600
1500,0.084100


TrainOutput(global_step=1875, training_loss=0.5475344614664713, metrics={'train_runtime': 2217.8107, 'train_samples_per_second': 6.758, 'train_steps_per_second': 0.845, 'total_flos': 5131571599245312.0, 'train_loss': 0.5475344614664713, 'epoch': 3.0})

In [14]:
# Save the fine-tuned model and tokenizer
trainer.save_model("./buffett_flan_model")
tokenizer.save_pretrained("./buffett_flan_model")

('./buffett_flan_model/tokenizer_config.json',
 './buffett_flan_model/special_tokens_map.json',
 './buffett_flan_model/spiece.model',
 './buffett_flan_model/added_tokens.json',
 './buffett_flan_model/tokenizer.json')

In [16]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from sentence_transformers import SentenceTransformer
import faiss
import pandas as pd

# Load dataset and embedding model
df = pd.read_csv("Dataset_Warren_Buffet_Clean.csv")
questions = df["question"].tolist()
answers = df["answer"].tolist()

embedder = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = embedder.encode(questions, convert_to_numpy=True)

# Build FAISS index
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

# Define retriever
def retrieve_context(query, k=1):
    query_embedding = embedder.encode([query], convert_to_numpy=True)
    _, indices = index.search(query_embedding, k)
    return answers[indices[0][0]]

# Choose a test query
query = "Why is having a margin of safety important?"
context = retrieve_context(query)

prompt = f"Context: {context}\n\nQuestion: {query}\nAnswer:"

# Load base and fine-tuned models
base_tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
base_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

finetuned_tokenizer = AutoTokenizer.from_pretrained("./buffett_flan_model")
finetuned_model = AutoModelForSeq2SeqLM.from_pretrained("./buffett_flan_model")

# Generate from base model
inputs_base = base_tokenizer(prompt, return_tensors="pt")
output_base = base_model.generate(**inputs_base, max_length=200)
answer_base = base_tokenizer.decode(output_base[0], skip_special_tokens=True)

# Generate from fine-tuned model
inputs_finetuned = finetuned_tokenizer(prompt, return_tensors="pt")
output_finetuned = finetuned_model.generate(**inputs_finetuned, max_length=200)
answer_finetuned = finetuned_tokenizer.decode(output_finetuned[0], skip_special_tokens=True)

# Print comparison
print("USER QUESTION:", query)
print("\n Retrieved Context:\n", context)
print("\n Base FLAN-T5 Answer:\n", answer_base)
print("\n Fine-Tuned BuffettBot Answer:\n", answer_finetuned)

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


USER QUESTION: Why is having a margin of safety important?

 Retrieved Context:
 In light of margin of safety, a margin of safety means buying assets at a discount to their intrinsic value to minimize the risk of loss

 Base FLAN-T5 Answer:
 minimize the risk of loss

 Fine-Tuned BuffettBot Answer:
 In light of margin of safety, a margin of safety means buying assets at a discount to their intrinsic value to minimize the risk of loss


# Streamlit BuffettBot

In [13]:
import streamlit as st

# Assume embedder, faiss_index, base_model, and buffett_model are pre-loaded and available.

st.title("BuffettBot Chat")

# Initialize chat history in session state if not already present
if "history" not in st.session_state:
    st.session_state.history = []

# Display the chat history
for entry in st.session_state.history:
    # Show user question
    st.markdown(f"**User:** {entry['user']}")
    # Show model answer with model name
    st.markdown(f"**{entry['model']}:** {entry['bot']}")
    # Expander to show retrieved context used for this answer
    with st.expander("Show retrieved context"):
        st.write(entry['context'])

# User controls: model selection and query input
model_choice = st.radio("Select model:", ("Base FLAN-T5", "Fine-Tuned BuffettBot"), key="model_choice")
user_question = st.text_input("Your question:", key="input_text")

# Handle submission when the user clicks the "Send" button
if st.button("Send") and user_question:
    # Retrieve relevant context from FAISS index using the embedder
    question_embedding = embedder.encode([user_question])
    top_k = 3  # number of context passages to retrieve
    distances, indices = faiss_index.search(question_embedding, top_k)
    # Gather retrieved text passages
    retrieved_texts = []
    for idx in indices[0]:
        # `documents` is a list or array of text entries corresponding to the FAISS index
        retrieved_texts.append(documents[idx])
    context_text = "\n\n".join(retrieved_texts)

    # Construct the prompt with context and question
    prompt = f"{context_text}\n\nQuestion: {user_question}\nAnswer:"

    # Generate answer using the selected model
    if model_choice == "Base FLAN-T5":
        result = base_model(prompt)
    else:
        result = buffett_model(prompt)
    # Extract the generated answer text (handle whether model returns a list or string)
    if isinstance(result, str):
        answer_text = result
    elif isinstance(result, list):
        # Assuming a HuggingFace pipeline output
        answer_text = result[0]['generated_text']
    else:
        # If using a model's generate method
        try:
            answer_text = result[0]['generated_text']
        except Exception:
            answer_text = str(result)

    # Append the new interaction to chat history
    st.session_state.history.append({
        "user": user_question,
        "bot": answer_text,
        "model": model_choice,
        "context": context_text
    })

    # Clear the input field for convenience
    st.session_state.input_text = ""

2025-08-05 01:07:06.749 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-05 01:07:06.751 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-05 01:07:06.755 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-05 01:07:06.756 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-05 01:07:06.759 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-05 01:07:06.760 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-05 01:07:06.764 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-05 01:07:06.766 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [15]:
with open("buffett_bot_app.py", "w") as f:
    f.write("""
import streamlit as st

# Assume embedder, faiss_index, documents, base_model, and buffett_model are pre-loaded and available

st.title("BuffettBot Chat")

# Initialize chat history in session state if not already present
if "history" not in st.session_state:
    st.session_state.history = []

# Display the chat history
for entry in st.session_state.history:
    # Show user question
    st.markdown(f"**User:** {entry['user']}")
    # Show model answer with model name
    st.markdown(f"**{entry['model']}:** {entry['bot']}")
    # Expander to show retrieved context used for this answer
    with st.expander("Show retrieved context"):
        st.write(entry['context'])

# User controls: model selection and query input
model_choice = st.radio("Select model:", ("Base FLAN-T5", "Fine-Tuned BuffettBot"), key="model_choice")
user_question = st.text_input("Your question:", key="input_text")

# Handle submission when the user clicks the "Send" button
if st.button("Send") and user_question:
    # Retrieve relevant context from FAISS index using the embedder
    question_embedding = embedder.encode([user_question])
    top_k = 3  # number of context passages to retrieve
    distances, indices = faiss_index.search(question_embedding, top_k)
    # Gather retrieved text passages
    retrieved_texts = []
    for idx in indices[0]:
        retrieved_texts.append(documents[idx])
    context_text = "\\n\\n".join(retrieved_texts)

    # Construct the prompt with context and question
    prompt = f"{context_text}\\n\\nQuestion: {user_question}\\nAnswer:"

    # Generate answer using the selected model
    if model_choice == "Base FLAN-T5":
        inputs = base_tokenizer(prompt, return_tensors="pt")
        outputs = base_model.generate(**inputs, max_length=200)
        answer_text = base_tokenizer.decode(outputs[0], skip_special_tokens=True)
    else:
        inputs = finetuned_tokenizer(prompt, return_tensors="pt")
        outputs = finetuned_model.generate(**inputs, max_length=200)
        answer_text = finetuned_tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Append the new interaction to chat history
    st.session_state.history.append({
        "user": user_question,
        "bot": answer_text,
        "model": model_choice,
        "context": context_text
    })

    # Clear the input field for convenience
    st.session_state.input_text = ""
""")

In [25]:
from pyngrok import conf

# Set your actual ngrok token
conf.get_default().auth_token = "NGROK_TOKEN"

In [ ]:
# Start Streamlit in background
!streamlit run buffett_bot_app.py &

# Give it some time to launch
import time
time.sleep(5)




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.233.208.147:8501



In [27]:
from pyngrok import ngrok

public_url = ngrok.connect(8501)
print(f"🔗 Your BuffettBot is live at: {public_url}")

🔗 Your BuffettBot is live at: NgrokTunnel: "https://f74eb93d39c7.ngrok-free.app" -> "http://localhost:8501"
